In [130]:
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import stopwords
from nltk import RSLPStemmer
from nltk import ngrams

from pandas import DataFrame, read_csv
from string import punctuation
from pathlib import Path

### Importa arquivo

In [131]:
news_data_path = Path('../arquivos/mongo_export.csv')
news_df = read_csv(news_data_path)

In [132]:
news_df.drop(columns='_id', inplace=True)

In [133]:
news_df.head()

,code,source,url,title,datetime,raw_text
0,BBAS3,Infomoney,https://www.infomoney.com.br/mercados/ceo-do-b...,CEO do Banco do Brasil diz que houve falha de ...,2021-02-12T10:41:35.000Z,André Brandão (Foto: Marcelo Camargo/ Agência ...
1,BBAS3,Infomoney,https://www.infomoney.com.br/mercados/despesas...,Despesas permanecerão sob rigoroso controle ne...,2021-02-12T07:10:00.000Z,André Brandão (Foto: Marcelo Camargo/ Agência ...
2,BBAS3,Infomoney,https://www.infomoney.com.br/mercados/bradesco...,"Bradesco, Itaú, Santander ou Banco do Brasil: ...",2021-02-12T17:41:41.000Z,SÃO PAULO – A temporada de resultados do quart...
3,BBAS3,Infomoney,https://www.infomoney.com.br/mercados/andre-br...,Ações do Banco do Brasil fecham em queda de 5%...,2021-01-13T18:11:02.000Z,"SÃO PAULO – As ações do Banco do Brasil (, R$ ..."
4,BBAS3,Infomoney,https://www.infomoney.com.br/mercados/banco-do...,Banco do Brasil anuncia abertura de 14 novas a...,2021-02-03T13:55:00.000Z,Seguindo o planejamento de reestruturação orga...


### Define funções para o tratamento dos textos

In [134]:
def tokenize(text: str) -> list:
    """ Divide the text into token """
    toknzr = nltk.tokenize.WhitespaceTokenizer()
    return toknzr.tokenize(text)

In [135]:
def remove_pt_stop_words(text: str) -> str:
    """ remove stop words from text in portuguese"""
    stop_words = nltk.corpus.stopwords.words("portuguese")
    no_stop_text = [word for word in text.split() if word not in stop_words]

    return ' '.join(no_stop_text)

In [136]:
def remove_punctuation(text: str) -> str:
    """ Remove punctuation from text """
    punctuation_list = ''.join([punctuation, chr(8211), chr(8220), chr(8221)])  # try regex
    punctuation_list = punctuation_list.replace('-', '')
    no_punct_text =  [word for word in text if word not in punctuation_list]
    no_punct_text = ''.join(no_punct_text)
    return no_punct_text

In [137]:
def stemmize(tokens: list) -> list:
    stemmer = nltk.RSLPStemmer()
    return [stemmer.stem(tok) for tok in tokens]

### Aplicação das funções no DataFrame

In [138]:
news_df['prep_text'] = news_df['raw_text'].apply(remove_punctuation)

In [139]:
news_df['prep_text'] = news_df['prep_text'].apply(remove_pt_stop_words)

In [140]:
news_df['prep_text'] = news_df['prep_text'].apply(tokenize)

In [141]:
news_df['prep_text'] = news_df['prep_text'].apply(stemmize)

In [142]:
news_df['prep_text'] = news_df['prep_text'].apply(lambda x : ' '.join(x))

In [143]:
news_df.head()

,code,source,url,title,datetime,raw_text,prep_text
0,BBAS3,Infomoney,https://www.infomoney.com.br/mercados/ceo-do-b...,CEO do Banco do Brasil diz que houve falha de ...,2021-02-12T10:41:35.000Z,André Brandão (Foto: Marcelo Camargo/ Agência ...,andré brand fot marcel camarg agênc brasilreut...
1,BBAS3,Infomoney,https://www.infomoney.com.br/mercados/despesas...,Despesas permanecerão sob rigoroso controle ne...,2021-02-12T07:10:00.000Z,André Brandão (Foto: Marcelo Camargo/ Agência ...,andré brand fot marcel camarg agênc brasil pre...
2,BBAS3,Infomoney,https://www.infomoney.com.br/mercados/bradesco...,"Bradesco, Itaú, Santander ou Banco do Brasil: ...",2021-02-12T17:41:41.000Z,SÃO PAULO – A temporada de resultados do quart...,são paul a tempor result quart trimestr 2020 a...
3,BBAS3,Infomoney,https://www.infomoney.com.br/mercados/andre-br...,Ações do Banco do Brasil fecham em queda de 5%...,2021-01-13T18:11:02.000Z,"SÃO PAULO – As ações do Banco do Brasil (, R$ ...",são paul as açõ banc brasil r 3755 -494 fech q...
4,BBAS3,Infomoney,https://www.infomoney.com.br/mercados/banco-do...,Banco do Brasil anuncia abertura de 14 novas a...,2021-02-03T13:55:00.000Z,Seguindo o planejamento de reestruturação orga...,segu planej reestrutur organizac anunci jan ba...


In [144]:
news_df.prep_text[0]

'andré brand fot marcel camarg agênc brasilreut o presidente-execu banc brasil andré brand afirm nest sexta-f 12 problem comunic presid jair bolsonar banc anunci fech 300 agênc tod paísbrand diss jornal bb abandon total nenhum cidad ele afirm banc decid fech agênc determin cidad outr agênc correspond banc fornec serviç básicosel reiter plan reduz númer agênc banc econom r 10 bilhã 2025'

In [146]:
news_df.to_csv(Path('../arquivos/BBAS3_prep.csv'), index=False)